# 유튜브 인기동영상 Data

In [2]:
import pandas as pd

# URL 형식으로 데이터 호출
dataUrl = 'https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv'
data = pd.read_csv(dataUrl,index_col = 0)
data.head()

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01


In [13]:
# 인기동영상 제작횟수 많은 채널 상위 10개 추출

data[data['channelId'].isin(data['channelId'].value_counts().head(10).index)].channelTitle.unique()

# data내에 channelId 값이 상위 10개의 id 값에 있는 값만 출력후, title 값으로 출력

array(['장삐쭈', '총몇명', '파뿌리', '짤툰', '런닝맨 - 스브스 공식 채널',
       '엠뚜루마뚜루 : MBC 공식 종합 채널', 'SPOTV', '채널 십오야', '이과장', 'BANGTANTV'],
      dtype=object)

In [17]:
# dislike수가 likes 수보다 높은 동영상 출력

data[data['dislikes']>data['likes']]['channelTitle'].unique()

array(['핫도그TV', 'ASMR 애정TV', '하얀트리HayanTree', '양팡 YangPang',
       '철구형 (CHULTUBE)', '왜냐맨하우스', '(MUTUBE)와꾸대장봉준', '오메킴TV', '육지담',
       'MapleStory_KR', 'ROAD FIGHTING CHAMPIONSHIP', '사나이 김기훈',
       '나혼자산다 STUDIO', 'Gen.G esports'], dtype=object)

In [36]:
# 채널명을 바꾼 케이스 ( channelId는 고유값)

data[['channelTitle','channelId']].duplicated().value_counts() # True 58556개 / False = 1843
chg = data[['channelTitle','channelId']].drop_duplicates().channelId.value_counts() 
print(len(chg[chg>1]))

71


In [45]:
# 특정요일에 인기있던 영상 카테고리

data.loc[:,'days']=pd.to_datetime(data['trending_date2']).dt.day_name() # 날짜형식으로 변환 후 요일별 이름 생성
data[data['days']=='Sunday']['categoryId'].value_counts().index[0] # 일요일에 해당하는 categoryId 값 카운트 후 호출

24

In [55]:
# 요일별 category 개수 출력

data.groupby(['categoryId','days']).size().unstack() # 데이터 프레임 형식으로 출력

days,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
categoryId,,,,,,,
1,243,263,255,274,246,257,234
2,120,105,119,99,128,119,129
10,833,837,776,830,890,894,917
15,187,215,198,217,207,208,207
17,633,668,592,636,682,708,706
19,90,92,87,91,92,89,85
20,283,298,296,289,282,285,291
22,1288,1373,1289,1337,1341,1375,1333
23,568,594,570,556,560,569,566


In [63]:
# viewcount 대비 댓글수가 가장 높은 영상 확인  (view_count값이 0은 제외)

tmp = data[data['view_count']!=0] # view_count 0값 제외
len(tmp[tmp['view_count']==0]) # result = 0

tmp.loc[:,'ratio'] = (tmp['comment_count']/tmp['view_count']).dropna() # viewcount대비 commennt_count 비율 값을 새로운 컬럼 생성
tmp.sort_values('ratio',ascending=False).iloc[0].title # ratio 값 기준으로 내림차순 정렬 후 첫번째 값 title 선택

C:\Users\SuperUser\AppData\Local\Temp\ipykernel_8840\2992716606.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.loc[:,'ratio'] = (tmp['comment_count']/tmp['view_count']).dropna() # viewcount대비 commennt_count 비율 값을 새로운 컬럼 생성


'60분 동안 댓글이 달리지 않으면, 영상이 삭제됩니다. (챌린지)'

In [68]:
# viewcount 대비 댓글수가 가장 낮은 영상 확인  (view_count,ratio값이 0은 제외)
len(tmp[tmp['ratio']==0]) # result = 777
tmp = tmp[tmp['ratio']!=0] # 0 값 제거
len(tmp[tmp['ratio']==0]) # result = 0

tmp.sort_values('ratio',ascending=True).iloc[0].title # 가장 ratio값이 낮은 title 출력

'Join the BTS #PermissiontoDance Challenge only on YouTube #Shorts'

In [76]:
# like 대비 dislike 수가 가작 적은 영상(0 값은 제외)

len(data[(data['likes']==0)|(data['dislikes']==0)]) # result = 706
tmp = data[(data['likes']!=0)&(data['dislikes']!=0)] # 0값 제외
len(tmp[(tmp['likes']==0)|(tmp['dislikes']==0)]) # result = 0

tmp.loc[:,'ratio'] = (tmp['dislikes']/tmp['likes']).dropna()
tmp.sort_values('ratio',ascending=True).iloc[0].title

C:\Users\SuperUser\AppData\Local\Temp\ipykernel_8840\2299923241.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.loc[:,'ratio'] = (tmp['dislikes']/tmp['likes']).dropna()


'[줌터뷰] *최초공개* 사부작즈🐰🐶의 비공식 이름은 아이라인즈? 꿀조합 티키타카 가득한 NCT 127 도영&정우의 줌터뷰'

In [92]:
# 가장많은 트랜드 영상 제작한 채널 이름

# data[data['channelId']=='UChbE5OZQ6dRHECsX0tEPEZQ']
data[['channelId','channelTitle']].value_counts().index[0][1] 

'짤툰'

In [98]:
# 20회 이상 인기동영상 리스트에 포함된 동영상

(data[['title','channelId']].value_counts()>=20).sum()

40

# 유튜브 공범컨텐츠 동영상 데이터

In [215]:
import pandas as pd


channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')
display(channel.head())
display(video.head())

,channelid,subcnt,viewcnt,videocnt,ct,channelname
0,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,410238653,736,2021-09-30 03:01:03,꽈뚜룹
1,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,736,2021-09-30 09:01:03,꽈뚜룹
2,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,735,2021-09-30 15:01:03,꽈뚜룹
3,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,737,2021-09-30 21:01:03,꽈뚜룹
4,UCkQCwnkQfgSuPTTnw_Y7v7w,1320000,412531322,737,2021-10-01 03:01:04,꽈뚜룹


,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
0,c5JQp6xafqc,1667010,30474,706,0,6587,2021-10-10 15:20:03,공범 EP1
1,c5JQp6xafqc,1669089,30495,707,0,6589,2021-10-10 15:30:03,공범 EP1
2,c5JQp6xafqc,1674759,30522,711,0,6596,2021-10-10 15:40:02,공범 EP1
3,c5JQp6xafqc,1677026,30555,712,0,6604,2021-10-10 15:50:03,공범 EP1
4,c5JQp6xafqc,1681824,30585,713,0,6600,2021-10-10 16:00:03,공범 EP1


In [216]:
# 날짜타입으로 변겨 후 video 데이터에서 videoname의 각 value 값 확인

channel['ct'] = pd.to_datetime(channel['ct'])
video['ct'] = pd.to_datetime(video['ct'])

video['videoname'].value_counts()

 공범 EP1    3492
 공범 EP2    3204
 공범 EP3    2568
 공범 EP4    2280
 공범 EP5    1562
 공범 EP6    1274
 공범 EP7     555
 공범 EP8     266
Name: videoname, dtype: int64

In [217]:
# 수집된 각 video의 가장 최신화 된 날짜의 viewcount 값 출력

tmp = video.drop_duplicates('videoname',keep='last')
tmp[['viewcnt','videoname','ct']].sort_values(['videoname','ct']).reset_index(drop=True)

,viewcnt,videoname,ct
0,3180532,공범 EP1,2021-11-01 15:30:03
1,2199328,공범 EP2,2021-11-01 15:30:03
2,1671294,공범 EP3,2021-11-01 15:30:03
3,1818493,공범 EP4,2021-11-01 15:30:03
4,1503435,공범 EP5,2021-11-01 15:30:04
5,1750222,공범 EP6,2021-11-01 15:30:04
6,1630200,공범 EP7,2021-11-01 15:30:05
7,1289088,공범 EP8,2021-11-01 15:30:05


In [218]:
# channel 데이터 중 2021-10-03일 이후 각 채널의 처음 기록 된 구독자 수 구하기

tmp = channel[channel['ct']>=pd.to_datetime('2021-10-03')].drop_duplicates('channelid',keep='first').sort_values(['ct','channelname']).reset_index(drop=True)
tmp[['channelname','subcnt']]

,channelname,subcnt
0,논리왕 전기,922000
1,야전삽짱재,257000
2,츄정ChuJeong,322000
3,김농밀의 농밀한 삶,7520
4,릴펄 Lilpearl,10100
5,와글와글 WagleWagle,55000
6,조나단,215000
7,형사!탐정되다,14900
8,Balming Tiger,54300
9,곽토리 kwak tori,471000


In [219]:
# 각 채널의 202-10-03 03:00:00~2021-11-01 15:00:00까지 구독자 수 증가량 구하기

end = channel[channel['ct'].dt.strftime('%Y-%m-%d %H')=='2021-11-01 15']
start = channel[channel['ct'].dt.strftime('%Y-%m-%d %H')=='2021-10-03 03']

end_data = end[['channelname','subcnt']].reset_index(drop=True)
start_data = start[['channelname','subcnt']].reset_index(drop=True)

end_data.columns = ['channelname','end_sub']
start_data.columns = ['channelname','start_sub']

m_data = pd.merge(start_data,end_data)
m_data.loc[:,'updown']  = m_data['end_sub']-m_data['start_sub']
m_data[['channelname','updown']]

,channelname,updown
0,꽈뚜룹,70000
1,야전삽짱재,11000
2,츄정ChuJeong,1000
3,논리왕 전기,-11000
4,와글와글 WagleWagle,0
5,조나단,12000
6,형사!탐정되다,10300
7,김농밀의 농밀한 삶,1540
8,릴펄 Lilpearl,11000
9,곽토리 kwak tori,-2000


In [220]:
# 수집간격 5분이하 20분 이상인 데이터 추출 ( 왜 실패??!!!)

tmp = video[video['videoname']==' 공범 EP1'].sort_values('ct').reset_index(drop=True)
tmp[(tmp['ct'].diff(1)>=datetime.timedelta(minutes=20))|(tmp['ct'].diff(1)<=datetime.timedelta(minutes=5))]

,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
721,c5JQp6xafqc,2228250,34559,849,0,7191,2021-10-13 09:41:37,공범 EP1
722,c5JQp6xafqc,2228250,34559,849,0,7191,2021-10-13 09:41:37,공범 EP1
1636,c5JQp6xafqc,2707933,37901,988,0,7410,2021-10-19 18:20:03,공범 EP1


In [221]:
# 각 에피소드 별 시작 날짜 및 이름 묶어서 데이터프레임 출력

tmp = video[['videoname','ct']].sort_values(['ct','videoname']).drop_duplicates('videoname',keep='first').reset_index(drop=True) # 전처리 순서 집중하기
tmp['date']=tmp['ct'].dt.date # dt.date = > 'YYYY-MM-DD' 형식으로 출력(문자열)
tmp[['videoname','date']]

,videoname,date
0,공범 EP1,2021-10-07
1,공범 EP2,2021-10-09
2,공범 EP3,2021-10-14
3,공범 EP4,2021-10-16
4,공범 EP5,2021-10-21
5,공범 EP6,2021-10-23
6,공범 EP7,2021-10-28
7,공범 EP8,2021-10-30


In [222]:
#“공범” 컨텐츠의 경우 19:00시에 공개 되는것으로 알려져있다. 공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 
#viewcnt를 내림차순으로 정렬하여 출력

tmp = video[video['ct'].dt.hour==21].sort_values(['ct','videoname']).drop_duplicates('videoname',keep='first')
tmp[['videoname','viewcnt','ct']].sort_values('viewcnt',ascending=False).reset_index(drop=True)

,videoname,viewcnt,ct
0,공범 EP8,264029,2021-10-30 21:00:08
1,공범 EP7,252032,2021-10-28 21:00:04
2,공범 EP4,217674,2021-10-16 21:00:04
3,공범 EP6,213899,2021-10-23 21:00:06
4,공범 EP5,201179,2021-10-21 21:00:04
5,공범 EP2,148144,2021-10-09 21:00:03
6,공범 EP3,147183,2021-10-14 21:00:04
7,공범 EP1,117340,2021-10-07 21:00:03


In [223]:
# video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고 
# videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차순으로 정렬

tmp = video.sort_values(['ct','videoname'],ascending=False).drop_duplicates('videoname',keep='first')
tmp['ratio'] = (tmp['dislikecnt']/tmp['likecnt']).dropna()

tmp[['videoname','ratio']].sort_values('ratio',ascending=True).reset_index(drop=True)

,videoname,ratio
0,공범 EP7,0.013636
1,공범 EP6,0.015881
2,공범 EP4,0.017971
3,공범 EP5,0.021128
4,공범 EP3,0.022360
5,공범 EP1,0.027118
6,공범 EP2,0.029844
7,공범 EP8,0.141357


In [224]:
#2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임 출력
def check(x):
    result = max(x) - min(x)
    return result

tmp = video[(video['ct']>=pd.to_datetime('2021-11-01 00:00:00'))&(video['ct']<=pd.to_datetime('2021-11-01 15:00:00'))].reset_index(drop=True)
tmp[['videoname','viewcnt']].groupby('videoname').agg(check)

,viewcnt
videoname,
공범 EP1,13298
공범 EP2,10300
공범 EP3,9927
공범 EP4,9824
공범 EP5,10824
공범 EP6,14141
공범 EP7,26949
공범 EP8,89147


In [228]:
# video 데이터 중에서 중복되는 데이터가 존재한다. 중복되는 각 데이터의 시간대와 videoname 출력

tmp = video[(video.duplicated())==True]
tmp[['videoname','ct']]

,videoname,ct
722,공범 EP1,2021-10-13 09:41:37
3927,공범 EP2,2021-10-13 09:41:37


# 월드컵 출전선수 골기록

In [321]:
data= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv')
data.head()

,Player,Goals,Years,Country
0,Miroslav Klose,16,2002-2006-2010-2014,Germany
1,Ronaldo,15,1998-2002-2006,Brazil
2,Gerd Muller,14,1970-1974,Germany
3,Just Fontaine,13,1958,France
4,Pele,12,1958-1962-1966-1970,Brazil


In [322]:
# 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 득점수 데이터프레임 형태 출력

data.groupby('Country').sum().sort_values('Goals',ascending=False).iloc[:5]

,Goals
Country,
Brazil,228
Germany,226
Argentina,135
Italy,123
France,115


In [323]:
# 주어진 전체기간동안 골득점을 한 선수가 가장 많은 나라 상위 5개 국가와 그 선수 숫자를 데이터 프레임 형식으로 출력하라

tmp = data.groupby('Country')['Player'].size().sort_values(ascending=False).head(5).to_frame()
tmp.columns=['cnt']
tmp

,cnt
Country,
Brazil,81
Germany,78
Italy,60
Argentina,59
France,58


In [324]:
# Years 컬럼은 년도 -년도 형식으로 구성되어있고, 각 년도는 4자리 숫자이다. 
#년도 표기가 4자리 숫자로 안된 케이스가 존재한다. 해당 건은 몇건인지 출력하라
cnt=0
for i in range(len(data['Years'].str.split('-'))):
    for j in range(len(data['Years'].str.split('-')[i])):
        if len(data['Years'].str.split('-')[i][j]) !=4:
            cnt+=1
print(cnt)

45


In [335]:
# Q23에서 발생한 예외 케이스를 제외한 데이터프레임을 df2라고 정의하고 데이터의 행의 숫자를 출력

data['yearLst'] = data.Years.str.split('-')

def checkFour(x):
    for value in x:
        if len(str(value)) != 4:
            return False
        
    return True
    
df2 = data[data.check ==True].reset_index(drop=True)
print(df2.shape[0])

1250


In [350]:
df2

,Player,Goals,Years,Country,check,yearLst,LenCup
0,Miroslav Klose,16,2002-2006-2010-2014,Germany,True,"[2002, 2006, 2010, 2014]",4
1,Ronaldo,15,1998-2002-2006,Brazil,True,"[1998, 2002, 2006]",3
2,Gerd Muller,14,1970-1974,Germany,True,"[1970, 1974]",2
3,Just Fontaine,13,1958,France,True,[1958],1
4,Pele,12,1958-1962-1966-1970,Brazil,True,"[1958, 1962, 1966, 1970]",4
...,...,...,...,...,...,...,...
1245,Josip Skoblar,1,1962,Yugoslavia,True,[1962],1
1246,Safet Susic,1,1982-1990,Yugoslavia,True,"[1982, 1990]",2
1247,Aleksandar Tirnanic,1,1930,Yugoslavia,True,[1930],1
1248,Djordje Vujadinovic,1,1930,Yugoslavia,True,[1930],1


In [342]:
# 월드컵 출전횟수를 나타내는 ‘LenCup’ 컬럼을 추가하고 4회 출전한 선수의 숫자를 구하여라

df2['LenCup']=df2['yearLst'].str.len()
result = df2['LenCup'].value_counts()[4]
result

16

In [355]:
# Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자를 구하여라
len(df2[(df2['Country']=='Yugoslavia')&(df2['LenCup']==2)])

7

In [363]:
#2002년도에 출전한 전체 선수는 몇명인가?

len(df2[df2['Years'].str.contains('2002')])

156

In [373]:
# 이름에 ‘carlos’ 단어가 들어가는 선수의 숫자는 몇 명인가? (대, 소문자 구분 x)

len(df2[df2['Player'].str.lower().str.contains('carlos')])

13

In [379]:
#월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수는 누구인가?

df2[['Player','Goals']][df2['LenCup']==1].iloc[0].Player

'Just Fontaine'

In [385]:
#월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가는 어디인가?

df2[df2['LenCup']==1]['Country'].value_counts().index[0]

'Brazil'